In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.models import Model, Sequential
import tensorboard

In [2]:
# Model 1: from research paper
# from the previous model, we got a traininig accuracy of about 82%. We feel that hair removal has distorted the
# training images. therefore, in this model, we try to train with data without performing hair removal to check 
# the training process.

In [3]:
batch_size = 32
img_height = 128
img_width = 128

In [4]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', padding= 'same', input_shape=(img_height, img_width, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu', padding= 'same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding= 'same'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=8, activation = 'softmax'))

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 64)       0

In [6]:
# load the augmented data from the local data directory:

train_dir = "Data/train_augmented"
test_dir = "Data/Processed_Data/test"
valid_dir = "Data/Processed_Data/valid"

datagen = ImageDataGenerator(rescale = 1./255) # The imported data will be normalized here.

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as training data

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as testing data

validation_generator = datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    seed=123,) # set as validation data

Found 72096 images belonging to 8 classes.
Found 3799 images belonging to 8 classes.
Found 3796 images belonging to 8 classes.


In [7]:
r = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 50,
)

C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_10608/3220806991.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
2253/2253 [==============================] - 177s 74ms/step - loss: 1.6283 - accuracy: 0.3847 - val_loss: 1.4999 - val_accuracy: 0.4486
Epoch 2/50
2253/2253 [==============================] - 154s 68ms/step - loss: 1.3972 - accuracy: 0.4795 - val_loss: 1.2944 - val_accuracy: 0.5384
Epoch 3/50
2253/2253 [==============================] - 163s 72ms/step - loss: 1.2803 - accuracy: 0.5265 - val_loss: 1.2320 - val_accuracy: 0.5440
Epoch 4/50
2253/2253 [==============================] - 153s 68ms/step - loss: 1.1986 - accuracy: 0.5555 - val_loss: 1.1504 - val_accuracy: 0.5887
Epoch 5/50
2253/2253 [==============================] - 145s 64ms/step - loss: 1.1384 - accuracy: 0.5805 - val_loss: 1.1755 - val_accuracy: 0.5760
Epoch 6/50
2253/2253 [==============================] - 150s 66ms/step - loss: 1.0952 - accuracy: 0.5950 - val_loss: 1.2578 - val_accuracy: 0.5344
Epoch 7/50
2253/2253 [==============================] - 142s 63ms/step - loss: 1.0538 - accuracy: 0.6125 - val_loss: 1

In [ ]:
#model.save('model_without_HR_2019.h5')

In [4]:
#model = tf.keras.models.load_model('model_without_HR_2019.h5')

In [7]:
#Confution Matrix and Classification Report
Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['ak', 'bcc', 'bkl', 'df', 'mel','nv', 'scc', 'vasc']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

C:\Users\GHAYAAS\AppData\Local\Temp/ipykernel_7388/2098712288.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model.predict_generator(test_generator, test_generator.samples // batch_size+1)


Confusion Matrix
[[ 10  18  14   4  26  54   1   3]
 [ 30  56  60   4 113 213  15   7]
 [ 26  58  45   3  87 149  18   8]
 [  6   4   3   0   8  14   1   0]
 [ 54 124  66  10 118 267  28  11]
 [162 284 208  26 396 776  53  26]
 [ 10  13  13   1  14  41   1   1]
 [  7   4   2   0   6  18   1   0]]
Classification Report
              precision    recall  f1-score   support

          ak       0.03      0.08      0.05       130
         bcc       0.10      0.11      0.11       498
         bkl       0.11      0.11      0.11       394
          df       0.00      0.00      0.00        36
         mel       0.15      0.17      0.16       678
          nv       0.51      0.40      0.45      1931
         scc       0.01      0.01      0.01        94
        vasc       0.00      0.00      0.00        38

    accuracy                           0.26      3799
   macro avg       0.11      0.11      0.11      3799
weighted avg       0.31      0.26      0.28      3799



In [8]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_generator.classes, y_pred))

0.2648065280336931


In [9]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(test_generator.classes, y_pred))

0.11126624861281842
